trail 1 
项目结构回顾：
    功能：实现单次微调或超参数优化(基于Optuna)
    数据集:
        基于datasets-cli并指定--reference参数下载了NCBI上Taxonomy是Fungi的genome file
        原始文件中显示5089条文件，但3811下载的路径下只找到2363条
        统计这批数据的metadata，我选择了这几个指标：Accession, GC content, Assembly level, Genome size，organism name
        以下代码用于获取这批数据的metadata:

        ```bash
        cat accession_paths.txt | awk -F '/' '{print $(NF)}' | cut -d '.' -f1 > accessions.txt
         datasets summary genome accession --inputfile accessions.txt --as-json-lines | dataformat tsv genome --fields accession,organism-name,assminfo-level,assmstats-gc-percent > genome_metadata.tsv
        ```
        由于datasets-cli获取genome size的参数不知道，所以用下面的shell脚本根据原始文件映射得到:
        

        ```bash
        awk 'BEGIN{OFS="\t"} NR==FNR{a[$1]=$NF; next} FNR==1{print $0, "Total Sequence Length"} {print $0, a[$1]}' \
         original_fungi_assembly_stats.tsv \
        genome_metadata.tsv > genome_metadata_with_length.tsv

        echo "合并完成，输出文件为 genome_metadata_with_length.tsv"
        ```
        这样就得到了目标指标的metadata文件，文件内容如下：

        ```bash
        Assembly Accession      Organism Name   Assembly Level  Assembly Stats GC Percent       Total Sequence Length
        GCA_000412225.2 [Ashbya] aceris (nom. inval.)   Complete Genome 51.5    8867527
        GCA_947297635.1 [Candida] aaseri        Contig  34      10658036
        GCA_030558135.1 [Candida] adriatica     Scaffold        52      10407487
        GCA_963924405.1 [Candida] anglica       Complete Genome 38.5    13719988
        GCA_030566955.1 [Candida] anutae        Scaffold        48      11255714
        GCA_001661425.1 [Candida] arabinofermentans NRRL YB-2248        Scaffold        34.5    13233932
        GCA_030561325.1 [Candida] atlantica     Scaffold        39      11581361
        GCA_030566905.1 [Candida] aurita        Scaffold        35.5    19613085
        ```

In [ ]:
import torch